In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from scipy import interpolate
from sklearn.model_selection import train_test_split
import lightgbm as lgb
from sklearn.metrics import mean_absolute_error
from statistics import mean 
from matplotlib import pyplot as plt
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

#!pip install autokeras

In [ ]:
!pip install pmdarima

In [ ]:
import pmdarima as pm
from pmdarima import datasets
from pmdarima import utils
from pmdarima import arima
from pmdarima import model_selection

参考:https://www.kaggle.com/code/tatsuyafujii/eda-jp-tokyo-stock-1

In [ ]:
fin_df=pd.read_csv("../input/jpx-tokyo-stock-exchange-prediction/train_files/financials.csv")

In [ ]:
prices=pd.read_csv("../input/jpx-tokyo-stock-exchange-prediction/supplemental_files/stock_prices.csv",parse_dates=["Date"])
print(prices.shape)
prices.head()

In [ ]:
#欠損値確認
print(prices.isnull().sum())
#データの型を確認
print(prices.dtypes)

591ってなんの数字？706?
Target 6欠損は何？
ExpectedDividendは一回除去する？

In [ ]:
#prices=prices.fillna(prices.median())

In [ ]:
#前の日の数値で欠損地を埋める
prices=prices.fillna(method="ffill")
#ExpectedDividendを消す
prices=prices.drop('ExpectedDividend', axis=1)

In [ ]:
print(prices.isnull().sum())

In [ ]:
prices.describe()

In [ ]:
#始値が欠損ということは出来高が0ということかどうか
(prices["Open"].isnull()==(prices["Volume"]==0)).all()

始値:ある期間の取引において、最初に成立した売買価格のこと
出来高:期間中に成立した売買の数量

In [ ]:
#それぞれの銘柄の数を数えると欠損が見えてくるはず.sore_valuesで昇順にしたので欠損がわかる
prices["SecuritiesCode"].value_counts().sort_values()

1413が4欠損、8806が3欠損。他はなし

In [ ]:
#欠損銘柄専用のデータフレームを作る(これ最後じゃなかったらどうするんだ...?)
pd.pivot(prices,index="Date",columns="SecuritiesCode",values="Close")[[1413,8806]]

Target:1日後と二日後の差額からえられる収益率。
securitiessCode:銘柄コード。2000種。

In [ ]:
#Targetの算出が合っているのか確認？

print(prices["Target"].head())
print(prices.groupby("SecuritiesCode")["Close"].apply(lambda s:(s.shift(-2)-s.shift(-1))/s.shift(-1)).head())

In [ ]:
#ほんとはもっと確認してるけどめんどいのでカット

In [ ]:
import matplotlib.pyplot as plt
x=prices.iloc[:,0]
y=prices.iloc[:,1]

plt.plot(x,y)
plt.show()

In [ ]:
#最終兵器AUtoML大作戦
#import autokeras

In [ ]:
import matplotlib.pyplot as plt
y=prices[prices["SecuritiesCode"]==1301]["Target"].rolling(window=3).mean()
x=prices[prices["SecuritiesCode"]==1301]["Date"]

plt.plot(x,y)
plt.show()

In [ ]:
prices[prices["SecuritiesCode"]==1301]["Target"]

In [ ]:
y=prices[prices["SecuritiesCode"]==1332]["Target"].rolling(window=3).mean()
x=prices[prices["SecuritiesCode"]==1332]["Date"]
plt.plot(x,y)
plt.show()

In [ ]:
prices[prices["SecuritiesCode"]==1302]

In [ ]:
y=prices[prices["SecuritiesCode"]==1333]["Target"].rolling(window=3).mean()
x=prices[prices["SecuritiesCode"]==1333]["Date"]
plt.plot(x,y)
plt.show()

In [ ]:
y=prices[prices["SecuritiesCode"]==1375]["Target"].rolling(window=3).mean()
x=prices[prices["SecuritiesCode"]==1375]["Date"]
plt.plot(x,y)
plt.show()

In [ ]:
#データ分割
#y=prices["Target"]
#X=prices.drop("Target", axis=1)
#X_train, X_test, y_train, y_test = train_test_split(X,
                                                    #y,
                                                    #train_size=0.75,
                                                    #test_size=0.25,
                                                    #random_state=42)

In [ ]:
from sklearn.tree import DesionTreeClassifier
import sklearn.model_selection

exit=prices.loc[prices["Target"]]

In [ ]:
prices.dtypes

ARIMAモデルチャレンジ

In [ ]:
plt.plot(prices[prices["SecuritiesCode"]==1375]["Target"])

In [ ]:
prices=prices.set_index("Date")
prices[prices["SecuritiesCode"]==1375]["Target"].head(50)

In [ ]:
utils.decomposed_plot(arima.decompose(prices[prices["SecuritiesCode"]==1375]["Target"],'additive',m=12))

In [ ]:
codes=prices["SecuritiesCode"]
codes=codes.unique()
len(codes)
#codes=list(codes)
#codes.unique()
#codes.apply(list)
#codes.values.tolist()
#print(list_sample)

In [ ]:
import datetime
SecuritiesCode, Date = np.meshgrid(codes,[(datetime(2021,12, 6) + timedelta(days=i)).strftime("%Y-%m-%d") for i in range(172)]
    )

Sec_date_df = pd.DataFrame(
        {
            "SecuritiesCode": SecuritiesCode.ravel(),
            "Date": Date.ravel(),
        }
    )

# 結合してソート
prices = pd.merge(Sec_date_df, prices, how="left").sort_values(["SecuritiesCode", "Date"])
# NaNを前の値で埋める
prices=prices.fillna(method="ffill")
# インデックのリセット
prices.reset_index(inplace=True, drop=True)

時系列->テーブル

In [ ]:
df=prices
import datetime

In [ ]:
# 年、月、日、曜日、祝日の列を追加
df['year'] = df.index.year
df['month'] = df.index.month
df['day'] = df.index.day
df['weekday'] = df.index.weekday
#df['is_holiday'] = df.index.map(jpholiday.is_holiday).astype(int)

In [ ]:
for j in range(1,2000):
    mean_values = []
    temp_df = df.loc[df.index <= datetime(2022,5,27)].copy()
    df_1=temp_df[temp_df["SecuritiesCode"] == j].copy()
    for i in range(7):
        mean_value = df_1[df_1['weekday']==i]['Target'].mean()
        mean_values.append(mean_value)

    df.loc[(df['weekday']==0) & (df['branch_id']==j),'mean_by_weekday'] = mean_values[0]
    df.loc[(df['weekday']==1) & (df['branch_id']==j),'mean_by_weekday'] = mean_values[1]
    df.loc[(df['weekday']==2) & (df['branch_id']==j),'mean_by_weekday'] = mean_values[2]
    df.loc[(df['weekday']==3) & (df['branch_id']==j),'mean_by_weekday'] = mean_values[3]
    df.loc[(df['weekday']==4) & (df['branch_id']==j),'mean_by_weekday'] = mean_values[4]
    df.loc[(df['weekday']==5) & (df['branch_id']==j),'mean_by_weekday'] = mean_values[5]
    df.loc[(df['weekday']==6) & (df['branch_id']==j),'mean_by_weekday'] = mean_values[6]